In [1]:
import os
import torch
import cv2
from torchvision import transforms
import numpy as np




In [2]:
# Define the transform
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert NumPy array to PIL Image
    transforms.Resize((112, 112)),  # Resize frame to 112x112
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Function to extract and preprocess frames
def prepare_video(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = torch.linspace(0, total_frames - 1, num_frames).long()

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx.item())
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frames.append(transform(frame))
    
    cap.release()

    # Stack frames and format for the model
    video_tensor = torch.stack(frames)  # Shape: [T, C, H, W]
    video_tensor = video_tensor.permute(1, 0, 2, 3)  # [T, C, H, W] -> [C, T, H, W]
    return video_tensor.unsqueeze(0)  # Add batch dimension: [C, T, H, W] -> [B, C, T, H, W]



In [13]:
# Path to the folder containing MP4 files
folder_path = 'aie\\aie'

# Create a list to store the tensors and labels
video_tensors = []
labels = []

# Label mapping: 0=alert, 1=assault, 2=non_assault
label_mapping = {
    'non': 0,
    'alert.mp4': 1,
    'assault.mp4': 2,
}

i=0
# Loop over the files in the folder
for filename in os.listdir(folder_path):
    print('iteration:',i)
    print('file:',filename)
    if filename.endswith('.mp4'):
        # Construct the full path of the video file
        video_path = os.path.join(folder_path, filename)
        
        # Get the label from the filename (e.g., '1_alert' -> 'alert')
        label_name = filename.split('_')[1]  # Extract label (e.g., 'alert')
        print('label:',label_name)
        label = label_mapping.get(label_name, -1)  # Map label name to integer (default -1 if not found)
        print('label_no:',label)
        # Preprocess the video and get the tensor
        video_tensor = prepare_video(video_path)
        
        # Append the tensor and label to the lists
        video_tensors.append(video_tensor)
        labels.append(label)
        i+=1
        print('\n')
        
  
        

# Convert the list of tensors to a single tensor
video_tensors = torch.cat(video_tensors, dim=0)  # Shape: [num_videos, C, T, H, W]
labels = torch.tensor(labels)  # Shape: [num_videos]

# Save the tensors and labels to disk
torch.save(video_tensors, 'video_tensors.pt')  # Save video tensors
torch.save(labels, 'labels.pt')  # Save labels

print("Data saved to disk.")

iteration: 0
file: 100_assault.mp4
label: assault.mp4
label_no: 2


iteration: 1
file: 100_non_assault.mp4
label: non
label_no: 0


iteration: 2
file: 101_assault.mp4
label: assault.mp4
label_no: 2


iteration: 3
file: 101_non_assault.mp4
label: non
label_no: 0


iteration: 4
file: 102_assault.mp4
label: assault.mp4
label_no: 2


iteration: 5
file: 102_non_assault.mp4
label: non
label_no: 0


iteration: 6
file: 103_assault.mp4
label: assault.mp4
label_no: 2


iteration: 7
file: 103_non_assault.mp4
label: non
label_no: 0


iteration: 8
file: 104_assault.mp4
label: assault.mp4
label_no: 2


iteration: 9
file: 104_non_assault.mp4
label: non
label_no: 0


iteration: 10
file: 105_assault.mp4
label: assault.mp4
label_no: 2


iteration: 11
file: 105_non_assault.mp4
label: non
label_no: 0


iteration: 12
file: 106_assault.mp4
label: assault.mp4
label_no: 2


iteration: 13
file: 106_non_assault.mp4
label: non
label_no: 0


iteration: 14
file: 107_assault.mp4
label: assault.mp4
label_no: 2


ite